# Практика 6: Атака по переносу (Transfer Attack) на модели ИИ

Выполнил: Асатрян Давид Робертович ББМО-01-23

# Цель:
Изучить концепцию атаки по переносу, где противоречивые примеры, созданные для одной модели, используются для атаки на другую модель. Это задание требует создания нескольких моделей, генерации противоречивых примеров для одной модели и проверки их на другой модели.

# Задачи:
Загрузить несколько моделей, обученных на датасете MNIST.
Изучить теоретические основы атаки по переносу.
Реализовать атаку FGSM на одну модель и проверить, как противоречивые примеры влияют на другую модель.
Оценить точность обеих моделей на противоречивых примерах и проанализировать переносимость атак.

Шаг 1: Загруска библиотек

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import numpy as np

Шаг 2: Загрузка данных и создание моделей

In [2]:
# Этап 1: Загрузка данных и создание моделей
# Загрузка набора данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация изображений
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Преобразование меток в формат one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Модель 1: Простая полносвязная нейронная сеть
model1 = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.fit(train_images, train_labels, epochs=2)
model1.save('mnist_model1.h5')

# Модель 2: Свёрточная нейронная сеть (CNN)
model2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=2)
model2.save('mnist_model2.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8804 - loss: 0.4297
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9633 - loss: 0.1277


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 39s 20ms/step - accuracy: 0.9053 - loss: 0.3177
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 21ms/step - accuracy: 0.9848 - loss: 0.0513


Шаг 3: Реализация атаки FGSM

In [3]:
# Функция для FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    perturbed_image = image + epsilon * np.sign(gradient)
    perturbed_image = np.clip(perturbed_image, 0, 1)  # Ограничение значений в диапазоне [0, 1]
    return perturbed_image

# Генерация противоречивых примеров
def generate_fgsm_adversarial(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape(1, 28, 28, 1), dtype=tf.float32)
        label = tf.convert_to_tensor(labels[i].reshape(1, -1), dtype=tf.float32)

        # Вычисление градиента
        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)

        gradient = tape.gradient(loss, image)
        adv_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(adv_image.reshape(28, 28))

    return np.array(adversarial_images)

# Создание противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_fgsm_adversarial(model1, test_images, test_labels, epsilon)


Шаг 4: Оценка результатов

In [4]:
# Оценка первой модели на противоречивых примерах
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels, verbose=0)
print(f'Точность первой модели на противоречивых примерах: {acc1:.4f}')

# Оценка второй модели на противоречивых примерах первой модели
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels, verbose=0)
print(f'Точность второй модели на противоречивых примерах от первой модели: {acc2:.4f}')


Точность первой модели на противоречивых примерах: 0.1566
Точность второй модели на противоречивых примерах от первой модели: 0.9564


Шаг 5: Дополнительное задание

In [5]:
# Генерация противоречивых примеров для второй модели
adversarial_images_model2 = generate_fgsm_adversarial(model2, test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)

# Оценка первой модели на противоречивых примерах, созданных второй моделью
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28, 28), test_labels, verbose=0)
print(f'Точность первой модели на противоречивых примерах от второй модели: {acc3:.4f}')

# Оценка второй модели на собственных противоречивых примерах
adversarial_images_model2 = adversarial_images_model2.reshape(-1, 28, 28, 1)
adversarial_images_model2 = adversarial_images_model2.astype('float32')

loss4, acc4 = model2.evaluate(adversarial_images_model2, test_labels, verbose=0)
print(f'Точность второй модели на собственных противоречивых примерах: {acc4:.4f}')


Точность первой модели на противоречивых примерах от второй модели: 0.9298
Точность второй модели на собственных противоречивых примерах: 0.6366


# Вывод
Результаты оценки точности моделей на противоречивых примерах предоставляют важную информацию о их устойчивости к атакам.

1.Точность первой модели на противоречивых примерах: 0.1566. Этот низкий показатель указывает на высокую уязвимость модели к атакам, что подтверждает эффективность FGSM атаки на неё. Первая модель не смогла правильно классифицировать противоречивые примеры, что свидетельствует о её слабой защите.

2.Точность второй модели на противоречивых примерах от первой модели: 0.9564. Этот высокий уровень точности показывает, что вторая модель оказалась достаточно устойчивой к противоречивым примерам, созданным для первой модели. Это может указывать на то, что вторая модель обладает лучшими характеристиками обобщения и способна справляться с некоторыми атаками, которые не были предусмотрены при её обучении.

3.Точность первой модели на противоречивых примерах от второй модели: 0.9298. Этот результат демонстрирует, что первая модель также проявила значительную устойчивость к противоречивым примерам, созданным для второй модели. Это подтверждает концепцию переносимости атак, где противоречивые примеры могут быть успешно использованы для атаки на другую модель.

4.Точность второй модели на собственных противоречивых примерах: 0.6366. Хотя вторая модель показала более высокую устойчивость к атакам по сравнению с первой моделью, её точность на собственных противоречивых примерах всё же ниже ожидаемого уровня. Это свидетельствует о том, что даже более сложные модели могут быть уязвимы к определённым типам атак.

# Общий вывод
Данные результаты подчеркивают важность разработки более защищённых моделей машинного обучения и необходимость дальнейшего изучения механизмов атак по переносу. Эффективность FGSM атак и их влияние на разные архитектуры моделей указывают на необходимость внедрения методов защиты и повышения устойчивости к подобным угрозам в будущем.